<center>
<img src="../../img/ml_theme.png">
# Дополнительное профессиональное <br> образование НИУ ВШЭ
#### Программа "Практический анализ данных и машинное обучение"
<img src="../../img/faculty_logo.jpg" height="240" width="240">
## Автор материала: старший преподаватель Факультета Компьютерных Наук НИУ ВШЭ <br> Кашницкий Юрий
</center>
Материал распространяется на условиях лицензии <a href="https://opensource.org/licenses/MS-RL">Ms-RL</a>. Можно использовать в любых целях, кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Занятие 10. Временные ряды. Блендинг моделей.</center>
## <center>Практика.  Смешивание случайного леса и логитической регресии для набора данных по идентификации пользователей. Решение</center>

In [1]:
import numpy as np
import pickle
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss

**Считаем данные по посещению 10 пользователями 4913 сайтов. Формат – разреженный, данные считываем  с помощью pickle (сериализация в Python).**

In [2]:
with open('../../data/web_sessions_10users.pkl', 'rb') as X_sparse_pickle:
    data = pickle.load(X_sparse_pickle)
with open('../../data/web_sessions_10users_ids.pkl', 'rb') as y_pickle:
    target = pickle.load(y_pickle)

In [3]:
data.shape, target.shape

((14061, 4913), (14061,))

**Посмотрим на 1 строку. 1 – число посещений сайта с индесом 1 в сессии из 10 сайтов, 2  – число посещений сайта с индесом 2 в сессии из 10 сайтов, далее опять 1 – число посещений сайта с индесом 3 и т.д. Большинство сайтов не посещены, поэтому разреженный формат тут уместен.**

In [4]:
data[0,:].todense()

matrix([[1, 2, 1, ..., 0, 0, 0]])

**Делим на обучающую и проверочную части.**

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.3, 
                                                     random_state=7, stratify=target)

**Обучите логистическую регрессию стохастическим градиентным спуском. У SGDClassifier укажите только параметры loss, n_jobs  и random_state=7.**

In [6]:
sgd = SGDClassifier(loss='log', n_jobs=-1, random_state=7)

In [7]:
sgd.fit(X_train, y_train,)

SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=-1,
       penalty='l2', power_t=0.5, random_state=7, shuffle=True, verbose=0,
       warm_start=False)

**Сделайте вероятностный прогноз (predict_proba) для проверочной выборки X_valid. Посмотрите, как он будет выглядеть (первые несколько штук).**

In [8]:
sgd_valid_pred_prob = sgd.predict_proba(X_valid)

**Посчитайте многоклассовую логистическую функцию ошибки на проверочной выборке (X_valid, y_valid)**

In [9]:
round(log_loss(y_valid, sgd_valid_pred_prob), 3)

1.155

**Посчитайте долю правильных ответов SGD на проверочной выборке (X_valid, y_valid), получив прогнозы из предсказанных вероятностей. Помогут методы numpy unique (посмотреть уникальные метки целевого класса) и argmax (определить, вероятность отнесения к какому из классов макисмальна). Разберитесь сначала с одной строкой, потом – со всей матрицей sgd_valid_pred_prob.**

In [10]:
round(float(accuracy_score(y_valid, 
                           np.unique(y_valid)[np.argmax(sgd_valid_pred_prob, axis=1)])), 3)

0.712

**Сделайте все то же для случайного леса. Оставьте только параметры n_jobs=-1, random_state=7. Остальные не будем настраивать.**

In [11]:
forest = RandomForestClassifier(n_jobs=-1, random_state=7)
forest.fit(X_train, y_train)
forest_valid_pred_prob  = forest.predict_proba(X_valid)
print(round(log_loss(y_valid, forest_valid_pred_prob), 3))
print(round(float(accuracy_score(y_valid, np.unique(y_valid)[np.argmax(forest_valid_pred_prob, axis=1)])), 3))

3.306
0.682


**Посчитайте log_loss и долю правильных ответов для следующего прогноза: 0.8 \* sgd_valid_pred_prob + 0.2 \* forest_valid_pred_prob. Для каждого примера такая предсказанная вероятность будет линейной комбинацией предсказанных вероятностей отнесения к тому или иному классу, сделанных SGD и лесом.**

In [12]:
mix_valid_pred_prob = 0.8 * sgd_valid_pred_prob + 0.2 * forest_valid_pred_prob
print(log_loss(y_valid, mix_valid_pred_prob))
print(accuracy_score(y_valid, np.unique(y_valid)[np.argmax(mix_valid_pred_prob, axis=1)]))

0.933948544171
0.723157146243


### Настройка коэффициентов в смеси

**Реализуйте функцию cross_val_predict_proba, которая работает как cross_val_predict, только возвращает предсказанные вероятности, а не метки. Функция принимает следующие аргументы:**

- estimator – объект Estimator Scikit-learn (классификатор или регрессор)
- X_train - матрица NumPy, соответсnвующая обучающей выборке
- y_train - вектор NumPy с ответами для объектов из X_train
- skf – объект KFold (или StratifiedKFold), задающий кросс-валидацию

In [13]:
def cross_val_predict_proba(estimator, X_train, y_train, skf):
    
    prediction = np.zeros([y_train.shape[0], len(np.unique(y_train))])
    for train_idx, test_idx in skf.split(X_train, y_train):
        X_train_part, X_test = X_train[train_idx, :], X_train[test_idx,:]
        y_train_part, y_test = y_train[train_idx], y_train[test_idx]
        
        estimator.fit(X_train_part, y_train_part)
        
        prediction[test_idx, :] = estimator.predict_proba(X_test)
        
    return(prediction)

**Зададим конкретный вид кросс-валидации – 5-кратную, с перемешиванием объектов.**

In [14]:
skf = StratifiedKFold(n_splits=5, random_state=7, shuffle=True)

**Сделайте "Оut-of-Fold" прогноз для SGD, посмотрите на качество (log_loss и долю правильных ответов) на кросс-валидации.**

In [15]:
sgd_oof_pred = cross_val_predict_proba(sgd, X_train, y_train, skf)

In [16]:
print(log_loss(y_train, sgd_oof_pred))
print(accuracy_score(y_train, np.unique(y_valid)[np.argmax(sgd_oof_pred, axis=1)]))

1.22253381372
0.712863239179


**Сделайте "Оut-of-Fold" прогноз для леса, посмотрите на качество (log_loss и долю правильных ответов) на кросс-валидации.**

In [17]:
forest_oof_pred = cross_val_predict_proba(forest, X_train, y_train, skf)

In [18]:
print(log_loss(y_train, forest_oof_pred))
print(accuracy_score(y_train, np.unique(y_valid)[np.argmax(forest_oof_pred, axis=1)]))

3.46442186517
0.683804104857


**Подберем веса в смеси SGD и леса. Будем перебирать значения веса, с которым в смесь входят прогнозы SGD, – $w_1$. Тогда у прогнозов, сделанных лесом, вес будет $w_2 = 1 - w_1$. **

In [19]:
def best_linear_mix(target, pred1, pred2):
    best_loss, best_w1 = np.inf, np.inf
    best_pred = pred1
    for w1 in np.linspace(0, 1, 20):
        curr_pred = w1 * pred1 + (1 - w1) * pred2
        curr_loss = log_loss(target, curr_pred)
        if curr_loss < best_loss:
            best_loss = curr_loss
            best_pred = curr_pred
            best_w1 = w1
    return best_loss, best_pred, best_w1

In [20]:
best_loss, best_pred, best_w1 = best_linear_mix(y_train, sgd_oof_pred, forest_oof_pred)

In [21]:
best_loss, best_w1

(0.95307324433555185, 0.63157894736842102)

In [22]:
accuracy_score(y_train, np.unique(target)[np.argmax(best_pred, axis=1)])

0.73328591749644378

In [23]:
mix_valid_pred_probs = best_w1 * sgd_valid_pred_prob + (1 - best_w1) * forest_valid_pred_prob

In [24]:
print(log_loss(y_valid, mix_valid_pred_probs))
print(accuracy_score(y_valid, np.unique(target)[np.argmax(mix_valid_pred_probs, axis=1)]))

0.918776839725
0.733823180849


**Подведите итог: каковы метрики качества SGD и леса по отдельности на кросс-валидации по (X_train, y_train) и на проверочной выборке (X_valid, y_valid)? Каковы те же метрики для смеси предсказанных вероятностей SGD и леса?**

<style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;}
.tg .tg-e3zv{font-weight:bold}
.tg .tg-hgcj{font-weight:bold;text-align:center}
.tg .tg-9hbo{font-weight:bold;vertical-align:top}
.tg .tg-amwm{font-weight:bold;text-align:center;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-e3zv"></th>
    <th class="tg-hgcj">SGD</th>
    <th class="tg-hgcj">RF</th>
    <th class="tg-hgcj">$w_1*SGD+(1-w_1)*RF$</th>
  </tr>
  <tr>
    <td class="tg-e3zv">CV log_loss</td>
    <td class="tg-hgcj"><center>1.22</td>
    <td class="tg-hgcj"><center>3.46<br></td>
    <td class="tg-hgcj"><center>0.95</td>
  </tr>
  <tr>
    <td class="tg-e3zv">Holdout log_loss</td>
    <td class="tg-hgcj"><center>1.15</td>
    <td class="tg-hgcj"><center>3.31</td>
    <td class="tg-hgcj"><center>.92</td>
  </tr>
  <tr>
    <td class="tg-e3zv">CV accuracy</td>
    <td class="tg-hgcj"><center>0.71</td>
    <td class="tg-hgcj"><center>0.68</td>
    <td class="tg-hgcj"><center>.73</td>
  </tr>
  <tr>
    <td class="tg-9hbo">Holdout accuracy</td>
    <td class="tg-amwm"><center>0.71</td>
    <td class="tg-amwm"><center>0.68</td>
    <td class="tg-amwm"><center>.73</center></td>
  </tr>
</table>